##**Importação das bibliotecas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/trab_IA/dataset_espectro.zip

Archive:  /content/drive/MyDrive/trab_IA/dataset_espectro.zip
   creating: dataset_espectro/
   creating: dataset_espectro/test/
   creating: dataset_espectro/test/Blues/
  inflating: dataset_espectro/test/Blues/108.png  
   creating: dataset_espectro/test/Classical/
  inflating: dataset_espectro/test/Classical/59.png  
   creating: dataset_espectro/test/Country/
  inflating: dataset_espectro/test/Country/93.png  
   creating: dataset_espectro/test/Eletronical/
  inflating: dataset_espectro/test/Eletronical/47.png  
   creating: dataset_espectro/test/Folk/
  inflating: dataset_espectro/test/Folk/75.png  
   creating: dataset_espectro/test/Hip Hop/
  inflating: dataset_espectro/test/Hip Hop/62.png  
   creating: dataset_espectro/test/Instrumental/
  inflating: dataset_espectro/test/Instrumental/6.png  
   creating: dataset_espectro/test/Jazz/
  inflating: dataset_espectro/test/Jazz/87.png  
   creating: dataset_espectro/test/Pop/
  inflating: dataset_espectro/test/Pop/33.png  
   creati

##**Criação da Rede Neural Convolucional**

###**Inicializando a CNN**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##Pré-processamento


In [ ]:
train_dir = "/content/dataset_espectro/train"
test_dir = "/content/dataset_espectro/test"
val_dir = "/content/dataset_espectro/val"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 77 images belonging to 11 classes.


In [ ]:
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 22 images belonging to 11 classes.


In [ ]:
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 11 images belonging to 11 classes.


In [ ]:
input_shape = [400,1000,3]
num_classes = len(train_gen.class_indices)

##O modelo em si


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=input_shape))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

###**Treinando a CNN e estimando no conjunto de testes**

In [ ]:
history = cnn.fit(train_gen, epochs=30, validation_data=val_gen)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 90s 36s/step - accuracy: 0.1189 - loss: 39.5148 - val_accuracy: 0.0909 - val_loss: 14.1094
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 83s 25s/step - accuracy: 0.0637 - loss: 16.3326 - val_accuracy: 0.0909 - val_loss: 2.4451
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 145s 35s/step - accuracy: 0.0491 - loss: 2.6196 - val_accuracy: 0.0909 - val_loss: 2.3553
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 35s/step - accuracy: 0.1622 - loss: 2.3842 - val_accuracy: 0.0909 - val_loss: 2.3484
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 86s 35s/step - accuracy: 0.1425 - loss: 2.3301 - val_accuracy: 0.0909 - val_loss: 2.2953
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 26s/step - accuracy: 0.1685 - loss: 2.1613 - val_accuracy: 0.2727 - val_loss: 2.2871
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 85s 26s/step - accuracy: 0.2581 - loss: 2.0095 - val_accuracy: 0.2273 - val_loss: 2.2197
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 140s 25s/step - accuracy: 0.3524 - loss: 1.7990 - val_accuracy: 0.2727 - val_loss: 2.1360
Epoc

In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_gen)
print(f"Perda nos dados de teste: {test_loss}")
print(f"Acurácia nos dados de teste: {test_accuracy}")

NameError: name 'cnn' is not defined

In [ ]:
# Processar a imagem
from tensorflow.keras.preprocessing import image
import numpy as np


test_image = image.load_img("/content/dataset_espectro/test/Eletronical/47.png", target_size=(400, 1000))  # Tamanho da imagem usado no treinamento
test_image = image.img_to_array(test_image)  # Converte a imagem para um array NumPy
test_image = np.expand_dims(test_image, axis=0)  # Adiciona uma dimensão para o batch
test_image = test_image

In [ ]:
result = cnn.predict(test_image)

print(f"Result: {result}")

predicted_class_index = np.argmax(result)  # Obtém o índice da classe com maior probabilidade
print(f"Predicted class index: {predicted_class_index}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Result: [[1.0000000e+00 0.0000000e+00 3.3637510e-26 2.7023976e-25 0.0000000e+00
  6.4744788e-27 4.5108618e-16 0.0000000e+00 4.1161588e-12 3.1795836e-29
  8.2470836e-11]]
Predicted class index: 0


In [ ]:
# Dicionário de classes (gerado no treinamento)
class_indices = {'rock': 0, 'jazz': 1, 'pop': 2, 'hip hop':3,'blues':4,'classical':5,'country':6,'eletronical':7,'folk':8,'instrumental':9,'soul':10}  # Substitua com as classes reais do treinamento
index_to_class = {v: k for k, v in class_indices.items()}  # Converte índice para nome da classe

# Obter o nome da classe prevista
prediction = index_to_class[predicted_class_index]

# Exibir o resultado
print(f"Predição: {prediction}")

Predição: rock


# Função para receber o gênero de uma música do dataset de acordo com seu ID

In [20]:
import pandas as pd

def check_gender_by_id(filename, id, genders) -> None | str:
    df = pd.read_excel(filename)

    genero = df.loc[df['id'] == id, 'genero']

    if not genero.empty:
        if genero.iloc[0] in genders:
            return genero.iloc[0]
        else:
            return None
    else:
        return "ID not found"

In [22]:
# Exemplo de uso
arquivo = '/content/generos.xlsx'
id_musica = 17  # Coloque o ID desejado
genders = ['Rock', 'Jazz', 'Pop', 'Classical', 'Country', 'Eletronical']
genero = check_gender_by_id(arquivo, id_musica, genders)
print(f'O gênero da música com ID {id_musica} é: {genero}')

O gênero da música com ID 17 é: Rock


In [23]:
def dividir_musicas_em_listas(arquivo_xlsx, generos_validos):
    # Carrega o arquivo Excel para um DataFrame
    df = pd.read_excel(arquivo_xlsx)

    # Dicionário para armazenar as músicas de cada gênero
    generos = {}

    # Itera por todas as músicas no arquivo
    for id_musica in df['id']:
        # Usa a função check_gender_by_id para obter o gênero da música
        genero = check_gender_by_id(arquivo_xlsx, id_musica, generos_validos)

        # Se o gênero for válido (não None e dentro da lista de gêneros válidos), adiciona o ID
        if genero and genero in generos_validos:
            if genero not in generos:
                generos[genero] = []
            generos[genero].append(id_musica)

    # Listas para armazenar as músicas de treino, validação e teste
    musicas_treino = []
    musicas_validacao = []
    musicas_teste = []

    # Agora, vamos dividir as músicas por gênero em treino, validação e teste de forma aleatória
    for genero, musicas in generos.items():
        # Embaralha as músicas para aleatoriedade
        random.shuffle(musicas)

        # Verifica se há pelo menos 30 músicas para dividir
        if len(musicas) >= 30:
            # Divide as músicas de forma aleatória
            treino = musicas[:21]
            validacao = musicas[21:27]
            teste = musicas[27:30]

            # Adiciona as listas para cada categoria
            musicas_treino.extend(treino)
            musicas_validacao.extend(validacao)
            musicas_teste.extend(teste)
        else:
            # Se houver menos de 30 músicas, podemos decidir o que fazer, mas aqui estamos ignorando as músicas
            pass

    # Retorna as 3 listas
    return musicas_treino, musicas_validacao, musicas_teste

In [24]:
import random
treino,validacao,teste = dividir_musicas_em_listas('/content/generos.xlsx',genders)

In [26]:
for x in treino:
  print(check_gender_by_id('/content/generos.xlsx',x,genders))

Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
